### Librerías utilizadas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

### Datos utilizados

Se importan los datos que resultaron del proceso de preparación de datos:

In [2]:
data = pd.read_csv('C:/Users/gusta/OneDrive/Documentos/Tesis/Datos/Datos_prep.csv') 

Se seleccionan las variables que serán utilizadas como variables dependientes:

In [3]:
var_predictivas = ['CC','VFA','CC_GRAS','CC_MUSC','CC_HUES','CC_AGUA','PAS',
                  'PAM','PESO','PAD','TALLA','GLU','IMC','TARGET_TRI']
X = data[var_predictivas] 
y = data['TRI'] 

### Separación de datos

Para evaluar mejor el rendimiento del modelo, es una buena estrategia dividir el conjunto de datos dos: un conjunto de entrenamiento (70%) y uno de prueba (30%).

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=1)

## Modelos supervisados para Regresión

En esta sección se llevará a cabo la construcción de los modelos supervisados de Machine Learning que realicen una predicción de cual sería el valor de los Triglicéridos del paciente. 

### Árbol de decisión

Se contruye un modelo de Árbol de Decisión usando Scikit-learn.

In [5]:
# Parámetros para el modelo:
param_dict_tr= dict(criterion=['mse','friedman_mse','mae'], splitter=['best','random'],
                 max_depth=range(2,30), max_features=range(2,14),
                 max_leaf_nodes=range(5,35), min_samples_leaf=[0.05])

# Se crea el árbol de decisión para regresión:
tr_reg = DecisionTreeRegressor()

# Autotuning del modelo:
#grid_tr = GridSearchCV(cv=5, estimator=tr_reg, n_jobs=-1, 
#                       param_grid=param_dict_tr)
#grid_tr.fit(X_train,y_train)
#tr_reg = grid_tr.best_estimator_

rsgrid_tr = RandomizedSearchCV(tr_reg, param_dict_tr, cv=20,
                               n_iter=40, n_jobs=-1)
rsgrid_tr.fit(X_train,y_train)
tr_reg = rsgrid_tr.best_estimator_

# Se entrena el árbol de decisión:
tr_reg = tr_reg.fit(X_train,y_train)

In [6]:
# Los mejores parámetros:
#grid_tr.best_estimator_
rsgrid_tr.best_estimator_

DecisionTreeRegressor(max_depth=13, max_features=13, max_leaf_nodes=26,
                      min_samples_leaf=0.05, splitter='random')

### Random Forest

Se contruye un modelo de Random Forest usando Scikit-learn.

In [7]:
# Parámetros para el modelo:
param_dict_rf= dict(n_estimators=range(50,150), criterion=['mse','mae'],
                 max_depth=range(20,30), max_features=range(2,14),
                 max_leaf_nodes=range(5,20), min_samples_leaf=[0.05])

# Se crea el Random Forest para regresión:
rf_reg = RandomForestRegressor()

#Autotuning del modelo:
#grid_rf = GridSearchCV(cv=5, estimator=rf_reg, n_jobs=-1, 
#                       param_grid=param_dict_rf)
#grid_rf.fit(X_train,y_train)
#rf_reg = grid_rf.best_estimator_

rsgrid_rf = RandomizedSearchCV(rf_reg, param_dict_rf, cv=20,
                               n_iter=40, n_jobs=-1)
rsgrid_rf.fit(X_train,y_train)
rf_reg = rsgrid_rf.best_estimator_

# Se entrena el Random Forest
rf_reg = rf_reg.fit(X_train,y_train)

In [8]:
# Los mejores parámetros:
#grid_rf.best_estimator_
rsgrid_rf.best_estimator_

RandomForestRegressor(max_depth=23, max_features=11, max_leaf_nodes=5,
                      min_samples_leaf=0.05, n_estimators=109)

### Gradient Boosting

Se contruye un modelo de Gradient Boosting usando Scikit-learn.

In [10]:
# Parámetros para el modelo:
param_dict_gb= dict(loss=['ls','lad','huber','quantile'], 
                    criterion=['friedman_mse','mse','mae'], 
                    learning_rate=[0.05,0.10,0.15,0.20,0.25,0.30,0.5,0.8,1],
                    max_depth=range(2,10), max_features=range(2,14),
                    max_leaf_nodes=range(5,35), min_samples_leaf=[0.05],
                    n_estimators=range(50,150))

# Se crea el Gradient Boosting para regresión:
gb_reg = GradientBoostingRegressor()

#Autotuning del modelo:
#grid_gb = GridSearchCV(cv=5, estimator=gb_reg, n_jobs=-1, 
#                       param_grid=param_dict_gb)
#grid_gb.fit(X_train,y_train)
#gb_reg = gb_reg.best_estimator_

rsgrid_gb = RandomizedSearchCV(gb_reg, param_dict_gb, cv=20,
                               n_iter=40, n_jobs=-1)
rsgrid_gb.fit(X_train,y_train)
gb_reg = rsgrid_gb.best_estimator_

# Se entrena el Gradient Boosting
gb_reg = gb_reg.fit(X_train,y_train)

In [11]:
# Los mejores parámetros:
#grid_gb.best_estimator_
rsgrid_gb.best_estimator_

GradientBoostingRegressor(criterion='mse', learning_rate=0.05, max_features=10,
                          max_leaf_nodes=27, min_samples_leaf=0.05,
                          n_estimators=134)

### XGBoost

Se contruye un modelo de Gradient Boosting usando Scikit-learn.

In [12]:
# Parámetros para el modelo:
param_dict_xgb= {'eta': [0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
                 'max_depth':[3,4,5,6,8,10,12,15],
                 'min_child_weight': [1,3,5,7],
                 'gamma': [0.0,0.1,0.2,0.3,0.4],
                 'colsample_bytree': [0.3,0.4,0.5,0.7],
                 'n_estimators': range(50,150)}

# Se crea el XGBoost para regresión:
xg_reg = XGBRegressor()

#Autotuning del modelo:
#grid_xgb = GridSearchCV(cv=5, estimator=xg_reg, n_jobs=-1, 
#                        param_grid=param_dict_xgb)
#grid_xgb.fit(X_train,y_train)
#xg_reg = grid_xgb.best_estimator_

rsgrid_xgb = RandomizedSearchCV(xg_reg, param_dict_xgb, cv=20,
                                n_iter=40, n_jobs=-1)
rsgrid_xgb.fit(X_train,y_train)
xg_reg = rsgrid_xgb.best_estimator_

# Se entrena el XGBoost:
xg_reg = xg_reg.fit(X_train,y_train)

In [13]:
# Los mejores parámetros:
#grid_xgb.best_estimator_
rsgrid_xgb.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, eta=0.1, gamma=0.3,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.100000001, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=125, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

### Estadísticos de precisión

A continuación, se muestran los estadísticos MSE y R2:

In [14]:
modelos = [{'label':'Decision Tree','modelo':tr_reg,},
           {'label':'Random Forest','modelo':rf_reg,},
           {'label':'Gradient Boosting','modelo':gb_reg,},
           {'label':'XGBoost','modelo':xg_reg,}]

for m in modelos:
    modelo = m['modelo']
    R2 =  modelo.score(X_test, y_test)
    y_pred = modelo.predict(X_test)
    print('%s R-squared: %0.2f' % (m['label'], R2))
    print("%s MSE: %.2f" % (m['label'],mean_squared_error(y_test,y_pred)))

Decision Tree R-squared: 0.61
Decision Tree MSE: 1373.16
Random Forest R-squared: 0.59
Random Forest MSE: 1447.82
Gradient Boosting R-squared: 0.61
Gradient Boosting MSE: 1375.20
XGBoost R-squared: 0.55
XGBoost MSE: 1580.89


Por último, se muestran los valores pronosticados por el modelo con mejor desempeño y se comparan con los valores reales. 

In [21]:
y_pred = pd.DataFrame(tr_reg.predict(X_test),columns=['TRI_pred'])
y_test.reset_index(inplace=True, drop=True)
datos_pronosticados = pd.concat([y_test,y_pred], axis=1)
datos_pronosticados

,TRI,TRI_pred
0,80,83.658730
1,47,80.680672
2,71,93.677686
3,181,209.818182
4,107,98.164706
...,...,...
523,75,80.680672
524,50,93.677686
525,105,93.677686
526,78,80.680672


In [22]:
writer = pd.ExcelWriter('Predicciones.xlsx', engine='xlsxwriter')
datos_pronosticados.to_excel(writer, sheet_name='Sheet1')
writer.save()